In [ ]:
# pytorch cnn tutorial 
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html 

#https://www.youtube.com/watch?v=pDdP0TFzsoQ

In [10]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms

import os 
import pandas as pd 
from torch.utils.data import Dataset, DataLoader
from skimage import io 

#set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyperparameters
in_channel = 3
num_classes = 10
learning_rate = 1e-3
batch_size = 2
num_epoch = 10

# 1. Load my custom  dataset of cats and dogs 


In [21]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform 
    
    def __len__(self):
        return len(self.annotations) 

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index,1]))

        if self.transform:
            image = self.transform(image)
            
        return(image, y_label)
    
#load data     
dataset = CatsAndDogsDataset(csv_file= "./data/cats_dogs/cats_dogs.csv", root_dir="./data/cats_dogs/cats_dogs_resized", transform= transforms.ToTensor())

train_set, test_set = torch.utils.data.random_split(dataset, [8, 2])

train_loader = DataLoader(dataset=train_set, batch_size= batch_size, shuffle =True)
test_loader = DataLoader(dataset=test_set, batch_size= batch_size, shuffle =True)


In [23]:
#model 
model = torchvision.models.googlenet(pretrained= True)
model.to(device)

#loss and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#train network 
for epoch in range(num_epoch):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device = device)
        targets = targets.to(device = device)

        #forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step 
        optimizer.step()

    print(f'cost at epoch {epoch} is {sum(losses)/len(losses)}')

#check accuracy on training to see how good our model is 
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y, in loader:
            x = x.to(device = device)
            y = y.to(device = device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions ==y).sum()
            num_samples += predictions.size(0)
        
        print(f'got {num_correct} / {num_samples} with acurracy {float(num_correct)/float(num_samples)* 100}')

    model.train()

print("Checking accuracy on Traiining set")
check_accuracy(train_loader, model)
print("Checking accuracy on testing set")
check_accuracy(test_loader, model)



cost at epoch 0 is 6.141992449760437
cost at epoch 1 is 3.591049313545227
cost at epoch 2 is 1.8197463154792786
cost at epoch 3 is 0.7450966238975525
cost at epoch 4 is 0.24358582770219073
cost at epoch 5 is 0.2635478121228516
cost at epoch 6 is 0.09788583242334425
cost at epoch 7 is 0.06618897896260023
cost at epoch 8 is 1.2081416198052466
cost at epoch 9 is 0.035605399461928755
Checking accuracy on Traiining set
got 8 / 8 with acurracy 100.0
Checking accuracy on testing set
got 1 / 2 with acurracy 50.0
